In [37]:
# environment: sceneGraphs_groundTruth_Replica

This notebook has been prepared to retrieve the ground truth on the segmentation in the Replica dataset.

**TODO**: the work has been completed but not checked! Something is probably to fix: the POT in the frl_apartment_0 (one of the first 6 objects) appears in a strange position

In [1]:
from plyfile import *
import numpy as np
import os
import open3d as o3d
import json
from scipy.spatial import KDTree
import csv
import pyviz3d.visualizer as viz

## Get the files with the segmentation

Get a folder containing, somehow, a mesh for each instance. Be aware that it seems that somehow each mesh still contains the information of the whole scene:

In [2]:
# Code from: https://github.com/facebookresearch/Replica-Dataset/issues/17

path_in_base = '/local/home/gmarsich/data2TB/DATASETS/Replica/frl_apartment_0/habitat/' #TODO TOSET
name = "mesh_semantic.ply"
path_in = os.path.join(path_in_base, name)

print("Reading input...")
file_in = PlyData.read(path_in)
vertices_in = file_in.elements[0]
faces_in = file_in.elements[1]

print("Filtering data...")
objects = {}
for f in faces_in:
     object_id = f[1]
     if not object_id in objects:
         objects[object_id] = []
     objects[object_id].append((f[0],))

print("Writing data...")
segmentation_dir = os.path.join(path_in_base, "Segmentation/")
os.makedirs(segmentation_dir, exist_ok=True)
for object_id, faces in objects.items():
    path_out = segmentation_dir + name + f"_{object_id}.ply"
    faces_out = PlyElement.describe(np.array(faces, dtype=[('vertex_indices', 'O')]), 'face')
    PlyData([vertices_in, faces_out]).write(path_out)

Reading input...
Filtering data...
Writing data...


Visualise an instance:

In [40]:
path_to_output_ply = '/local/home/gmarsich/data2TB/DATASETS/Replica/frl_apartment_0/habitat/Segmentation/mesh_semantic.ply_47.ply' #TODO TOSET

mesh = o3d.io.read_triangle_mesh(path_to_output_ply)

if not mesh.has_triangles():
    raise ValueError(f"Failed to load mesh from {path_to_output_ply}")

mesh.compute_vertex_normals()

# Visualize the mesh
o3d.visualization.draw_geometries([mesh])

## Get useful information

Get a `list list_file_paths` containing the paths of all the meshes (one mesh represents one instance). Ordered following the numbering of the meshes:

In [41]:
all_items = os.listdir(segmentation_dir)
list_file_paths = [os.path.join(segmentation_dir, item) for item in all_items if os.path.isfile(os.path.join(segmentation_dir, item))]
list_file_paths.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))

print(len(list_file_paths))


233


Create `list_points`. Each element is a list of points, there is a correspondance with `list_file_paths`:

In [42]:
np.random.seed(42)

list_points = [] # list_points[i] contains the points related to list_file_paths[i] (i.e., mesh_semantic.ply_i.ply). i represents the obj_id

# for i in range(len(list_file_paths)): #TODO: adjust depending on which instance you want to see
for i in range(6): #TODO: adjust depending on which instance you want to see
    path_to_mesh_ply = list_file_paths[i]
    mesh = o3d.io.read_triangle_mesh(path_to_mesh_ply)
    mesh.compute_vertex_normals()
    # Sample points from the mesh
    point_cloud = mesh.sample_points_uniformly(number_of_points=1000)  #TODO TOSET: adjust the number of points as needed
    list_points.append(np.asarray(point_cloud.points))


[Open3D WARNING] Read PLY failed: A polygon in the mesh could not be decomposed into triangles.
[Open3D WARNING] Read PLY failed: unable to read file: /local/home/gmarsich/data2TB/DATASETS/Replica/frl_apartment_0/habitat/Segmentation/mesh_semantic.ply_0.ply


RPly: Aborted by user


[Open3D WARNING] Read PLY failed: A polygon in the mesh could not be decomposed into triangles.
[Open3D WARNING] Read PLY failed: unable to read file: /local/home/gmarsich/data2TB/DATASETS/Replica/frl_apartment_0/habitat/Segmentation/mesh_semantic.ply_2.ply


RPly: Aborted by user


[Open3D WARNING] Read PLY failed: A polygon in the mesh could not be decomposed into triangles.
[Open3D WARNING] Read PLY failed: unable to read file: /local/home/gmarsich/data2TB/DATASETS/Replica/frl_apartment_0/habitat/Segmentation/mesh_semantic.ply_4.ply


RPly: Aborted by user


[Open3D WARNING] Read PLY failed: A polygon in the mesh could not be decomposed into triangles.
[Open3D WARNING] Read PLY failed: unable to read file: /local/home/gmarsich/data2TB/DATASETS/Replica/frl_apartment_0/habitat/Segmentation/mesh_semantic.ply_5.ply


RPly: Aborted by user


Create `list_info` with information for each object (`[obj_id, class_name, center]`). **Refer to the id when searching for something**:

In [43]:
# TODO: an idea could be to add a different color for each obj_id
# TODO: get the segmentation point cloud. But in that case we should choose the colors to give for each obj_id. They may also be random

name_semantic = "info_semantic.json"
path_info_semantic = os.path.join(path_in_base, name_semantic)

with open(path_info_semantic, 'r') as file:
    data = json.load(file)

objects = data.get('objects', [])
list_info = [] # remark that it will not be ordered

for obj in objects:
    obj_id = obj.get('id')
    class_name = obj.get('class_name')
    center = obj.get('oriented_bbox', {}).get('abb', {}).get('center', [])
    list_info.append([obj_id, class_name, center])

print(len(list_info))

226


**TODO**: Be aware that we may have that `len(list_file_paths) != len(list_info)`, but I don't really know why. Some ids seem to be missing in the `.json` file

## Get `matrix_distances`

Let's define some useful functions and the possible metrics to compute the distance between instances, and then let's get the results:

In [44]:
#
# Possible distance metrics
#

def distance_Euclidean_centerBoundingBoxes(center_1, center_2):
    distance = np.linalg.norm(center_1 - center_2)
    return distance


def distance_Euclidean_closest_points(list_points_1, list_points_2):
    tree = KDTree(list_points_2)
    min_distance = np.inf
    for point1 in list_points_1:
        dist, _ = tree.query(point1)
        if dist < min_distance:
            min_distance = dist
    return min_distance


#
# Useful functions
#

def get_list_instances(list_info, list_points):
    list_instances = [] # will contain a list of [obj_id, class_name, center, points]

    for obj in list_info:
        obj_id = obj[0]
        class_name = obj[1]
        center = obj[2]
        if obj_id < len(list_points):
            list_instances.append([obj_id, class_name, np.array(center), list_points[obj_id]])        
    
    transposed_list_instances = [list(row) for row in zip(*list_instances)]

    return list_instances, transposed_list_instances


def compute_distance_matrix(list_instances, compute_distance):
    matrix_distances = np.full((len(list_instances), len(list_instances)), np.inf)

    for i in range(len(matrix_distances)):
        for j in range(i + 1, len(matrix_distances[0])): # the matrix is symmetric

            if compute_distance == distance_Euclidean_centerBoundingBoxes:
                matrix_distances[i][j] = compute_distance(list_instances[i][2], list_instances[j][2])
            else:
                matrix_distances[i][j] = compute_distance(list_instances[i][3], list_instances[j][3])
            
            matrix_distances[j][i] = matrix_distances[i][j]

    # Save matrix_distances in a file
    with open("matrix_distances_file.txt", 'w') as file_matrix:
        file_matrix.write("\n")
        np.savetxt(file_matrix, matrix_distances, fmt='%.18e')

    # Save list_instances (but just the list of [obj_id, class_name, center], so without the list_points)
    with open("list_objects.txt", 'w') as file_objects:
        list_instances_noPoints = [sublist[:-1] for sublist in list_instances]
        for sublist in list_instances_noPoints:
            obj_id, class_name, center = sublist
            center_str = ', '.join(f'{coord:.6f}' for coord in center)
            file_objects.write(f"{obj_id}\t{class_name}\t{center_str}\n")
    
    return matrix_distances

In [45]:
list_instances, transposed_list_instances = get_list_instances(list_info, list_points)

matrix_distances = compute_distance_matrix(list_instances, compute_distance = distance_Euclidean_centerBoundingBoxes) # TODO TOSET: change the distance metric you want to use

## Get the scene graph

In [28]:
# TODO: automate the naming of the folder where scene graph are saved

### Version using PyViz3D

In [27]:
# TODO put in the Control panel the possibility to see and not to see the distances on the edges
# TODO change reference system (e.g., the origin is the centroid of the centroids)

Here I use the `PyViz3D` package (https://github.com/francisengelmann/PyViz3D), taking inspiration from this example: https://github.com/francisengelmann/PyViz3D/blob/master/examples/example_point_clouds.py to add the point cloud to the visualisation. Then, also vertices and edges have to be added.

Create a function that generates the scene graph:

In [46]:
threshold = 8 # TODO TOSET: distance threshold in meters

def create_sceneGraph(list_instances, matrix_distances, threshold, custom_color=[0, 0, 0]):
    # custom_color gives the color for vertices and edges
    vertices = [label[2] for label in list_instances] # the vertices are the centers of the bounding boxes

    # Create lines for the edges and save the associated distances
    edges = []
    distances_str = []
    distances = []
    for i in range(len(matrix_distances)):
        for j in range(i + 1, len(matrix_distances[0])):
            if matrix_distances[i][j] <= threshold:
                    edges.append([i, j])
                    distances.append(matrix_distances[i][j])
                    distances_str.append(f"{matrix_distances[i][j]:.2f}")

    lines_start = np.array([vertices[i] for (i, j) in edges])
    lines_end = np.array([vertices[j] for (i, j) in edges])
    midpoints = (lines_start + lines_end) / 2

    lines_colors = np.array([custom_color for (i, j) in edges])

    return lines_start, lines_end, lines_colors, [np.array(row) for row in midpoints], distances, distances_str

Visualise and save the scene graph:

In [47]:
v = viz.Visualizer()

pcd = o3d.io.read_point_cloud(os.path.join("/local/home/gmarsich/data2TB/DATASETS/Replica/frl_apartment_0/mesh.ply")) # TODO TOSET: change the name of the point cloud to open

name = '3D segmentation'
point_positions = np.asarray(pcd.points)
point_colors = (np.asarray(pcd.colors) * 255).astype(np.uint8)
point_size = 15 # TODO TOSET

# Here we add point clouds to the visualiser
v.add_points(name, point_positions, point_colors, point_size=point_size, visible=False)

colors = [np.array([0, 0, 0]) for label in transposed_list_instances[0]] # TODO TOSET

v.add_labels(name ='Labels',
                 labels = [label.upper() for label in transposed_list_instances[1]], # TODO TOSET: if you want in capital letters or not
                 positions = transposed_list_instances[2],
                 colors = colors,
                 visible=True)

lines_start, lines_end, lines_colors, midpoints, _, distances_str = create_sceneGraph(list_instances, matrix_distances, threshold, custom_color=[0, 0, 0])

v.add_lines(name='Edges', lines_start=lines_start, lines_end=lines_end, colors=lines_colors, visible=True)

# The following is to have the distances among instances
distances_on = True #TODO TOSET
if distances_on:
    v.add_labels(name ='Distances',
                    labels = distances_str,
                    positions = midpoints,
                    colors = [0, 0, 0] * len(midpoints), # TODO TOSET change the color of the text of distances
                    visible=True)

# When we added everything we need to the visualizer, we save it.
v.save('sceneGraph_PyViz3D')



************************************************************************
1) Start local server:
    cd /local/home/gmarsich/Desktop/Thesis/0Code_playground/sceneGraph_PyViz3D; python -m http.server 6008
2) Open in browser:
    http://localhost:6008
************************************************************************
